# [2] Conversations and Bookings

In [1]:
cd ..

/home/jovyan/Business_Analytics/Business-Insights-and-Analytics/Marketing_Analytics


In [2]:
%run __init__.py

## 1. What are the possible services and what is the average price per unit for each service type?

a) Use a SQL query to find the distinct service types and average price from the `services_service` table


In [3]:
service_types = list(services_service.service_type.unique())
service_types

['boarding', 'dog-walking', 'day-care']

In [4]:
# conn = sqlite3.connect('db26.sqlite3')

In [5]:
query_average_price = ('SELECT DISTINCT(service_type), AVG(price) as avg_price \
                        FROM services_service \
                        GROUP BY service_type;')

In [6]:
average_price_per_service = pd.read_sql_query(query_average_price, conn)

In [7]:
average_price_per_service

,service_type,avg_price
0,boarding,34.506077
1,day-care,27.013689
2,dog-walking,21.982185


## 2. How many requests have there been for each service type and what percentage of those have booked? The percentage of those that have booked is called booking rate.

a) Use SQL query to find the number of requests and number of bookings, grouping by the service type

b) In Python, calculate the booking rate as the `number_booked` divided by `number_requests`, multiply by 100

In [8]:
query_number_requests = ('SELECT DISTINCT(service_type), COUNT(requester_id) as number_requests, COUNT(booked_at) as number_booked \
                          FROM services_service s \
                          JOIN conversations_conversation c \
                          ON (s.id = c.service_id) \
                          GROUP BY service_type;')

In [9]:
requests_per_service = pd.read_sql_query(query_number_requests, conn)

In [10]:
requests_per_service['booking_rate'] = (requests_per_service.number_booked / 
                                        requests_per_service.number_requests) * 100

In [11]:
requests_per_service

,service_type,number_requests,number_booked,booking_rate
0,boarding,26665,10862,40.735046
1,day-care,26362,9417,35.721872
2,dog-walking,26716,12811,47.952538


## 3. What are the cancellation rates for each service?

a) Augment SQL query from above to include the number of cancellations per service type

b) Calculate cancellation rate by dividing `number_cancelled` by `number_booked`, multiply by 100

c) Merge tables together and format with the `service_type` as index

In [12]:
query_cancellations = (
    'SELECT DISTINCT(service_type), COUNT(requester_id) as number_requests, COUNT(booked_at) as number_booked, COUNT(cancelled_at) as number_cancelled \
     FROM services_service s \
     JOIN conversations_conversation c \
     ON (s.id = c.service_id) \
     GROUP BY service_type;'
)

In [13]:
cancellation_rate = pd.read_sql_query(query_cancellations, conn)

In [14]:
# cancellation rate: can only cancel an already booked request 
cancellation_rate['cancellation_rate'] = (cancellation_rate.number_cancelled / 
                                          cancellation_rate.number_booked) * 100

In [15]:
booking_cancellation_rates = pd.merge(cancellation_rate, requests_per_service, on=['service_type', 'number_requests', 'number_booked'])

In [16]:
rates_price_by_type = pd.merge(booking_cancellation_rates, average_price_per_service, on='service_type')
rates_price_by_type

,service_type,number_requests,number_booked,number_cancelled,cancellation_rate,booking_rate,avg_price
0,boarding,26665,10862,1051,9.675934,40.735046,34.506077
1,day-care,26362,9417,919,9.758947,35.721872,27.013689
2,dog-walking,26716,12811,1218,9.507455,47.952538,21.982185


In [17]:
rates_price_by_type.rename(columns={'service_type': 'Service Type',
                                    'number_requests': 'Number of Requests',
                                    'number_booked': 'Number Booked',
                                    'number_cancelled': 'Number Cancelled',
                                    'cancellation_rate': 'Cancellation Rate',
                                    'booking_rate': 'Booking Rate',
                                    'avg_price': 'Average Price'}, inplace=True)

In [18]:
rates_price_by_type = rates_price_by_type.set_index('Service Type')
rates_price_by_type

,Number of Requests,Number Booked,Number Cancelled,Cancellation Rate,Booking Rate,Average Price
Service Type,,,,,,
boarding,26665,10862,1051,9.675934,40.735046,34.506077
day-care,26362,9417,919,9.758947,35.721872,27.013689
dog-walking,26716,12811,1218,9.507455,47.952538,21.982185


## 4. For uncancelled bookings, is the owner or provider more likely to leave a review and which tends to leave better reviews? How would you narrate this finding to a business partner?

In [19]:
conversations_conversation.rename(columns={'id': 'conversation_id'}, inplace=True)

In [20]:
conversations_conversation.head(2)

,conversation_id,start_date,end_date,units,added,booking_total,cancellation_fault,requester_id,service_id,booked_at,cancelled_at
0,208294,2017-06-06,2017-06-09,3,2017-05-25 17:06:35.796792,102,NaN,64416,81642,2017-05-26 21:02:09.499404,NaN
1,208295,2017-07-16,2017-07-25,9,2017-07-07 06:07:08.053227,324,NaN,64415,79559,NaN,NaN


In [21]:
conversations_review.head(2)

,id,content,stars,conversation_id,reviewer_id
0,73828,Curae donec quam quisque quam.,4,208294,2617
1,73829,Risus donec scelerisque ultrices commodo nulla.,5,208296,64414


In [22]:
reviews = pd.merge(conversations_conversation, conversations_review, on='conversation_id')
reviews.head(2)

,conversation_id,start_date,end_date,units,added,booking_total,cancellation_fault,requester_id,service_id,booked_at,cancelled_at,id,content,stars,reviewer_id
0,208294,2017-06-06,2017-06-09,3,2017-05-25 17:06:35.796792,102,NaN,64416,81642,2017-05-26 21:02:09.499404,NaN,73828,Curae donec quam quisque quam.,4,2617
1,208296,2017-06-06,2017-06-12,6,2017-05-26 17:47:38.312937,102,NaN,64414,91348,2017-05-29 01:27:14.256611,NaN,73829,Risus donec scelerisque ultrices commodo nulla.,5,64414


In [23]:
reviews.drop(['start_date',
              'end_date',
              'units',
              'added',
              'booking_total',
              'cancellation_fault',
              'content'], axis=1, inplace=True)
reviews.head(2)

,conversation_id,requester_id,service_id,booked_at,cancelled_at,id,stars,reviewer_id
0,208294,64416,81642,2017-05-26 21:02:09.499404,NaN,73828,4,2617
1,208296,64414,91348,2017-05-29 01:27:14.256611,NaN,73829,5,64414


In [24]:
reviews.shape

(28536, 8)

In [25]:
# Checking to make sure that none of these entries have been cancelled. 
reviews.cancelled_at.value_counts(dropna=False)

NaN    28536
Name: cancelled_at, dtype: int64

In [26]:
reviews['who_booked'] = np.where(reviews['requester_id'] == reviews['reviewer_id'], 'Owner', 'Provider')
reviews.head(2)

,conversation_id,requester_id,service_id,booked_at,cancelled_at,id,stars,reviewer_id,who_booked
0,208294,64416,81642,2017-05-26 21:02:09.499404,NaN,73828,4,2617,Provider
1,208296,64414,91348,2017-05-29 01:27:14.256611,NaN,73829,5,64414,Owner


In [27]:
reviews.drop(['service_id',
              'booked_at',
              'cancelled_at',
              'id'], axis=1, inplace=True)
reviews.head(2)

,conversation_id,requester_id,stars,reviewer_id,who_booked
0,208294,64416,4,2617,Provider
1,208296,64414,5,64414,Owner


In [28]:
# Number in each category leaving reviews, owners leave more reviews than providers
reviews.groupby('who_booked')['stars'].count()

who_booked
Owner       22493
Provider     6043
Name: stars, dtype: int64

In [29]:
(reviews.groupby('who_booked')['stars'].sum()) / (reviews.groupby('who_booked')['stars'].count())

who_booked
Owner       4.405815
Provider    3.885818
Name: stars, dtype: float64

In [30]:
6043 / (6043+22493) * 100

21.176759181384917

In [31]:
22493 / (6043+22493) * 100

78.82324081861508

## Analysis

### Owners tend to leave better reviews than providers. One reason may be that reviews owners leave are shared with the provider, whereas reviews providers leave are private. 

### Owners also leave more reviews. 79% of the reviews come from owners, whereas only 21% come from providers. Owners care a lot about their animals and are highly invested in the service. I would be curious to know how many providers rely on this service as their soul source of income. I bet that many providers do not rely on this and so are not as invested as the owners are.

# 5. Further Questions
- Is there a correlation between the review statistics and longer stays, multiple dogs, weekend stays, or provider owning a dog themself?
- Where are the outliers in the ratings? 
- How does the company encourage owners and providers to write a review, is there a difference between the two groups.
- What is the average booking rate for owners/providers that leave reviews, is it different from owners/providers that do not leave reviews?
- Are there people that always leave a review or never leave a review?
- What percentage of bookings are reviewed?
- Are owners/providers more likely to leave a review if they liked a service?